In [ ]:
import pandas as pd
import requests
import json

from google.colab import drive
from io import StringIO

In [ ]:
drive.mount('/content/drive')
config_file_path = "/content/drive/MyDrive/Skrypty/config.json"

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

running = config['running_monthly_summary']
cycling = config['cycling_monthly_summary']

output_file = config['workouts_summary_export']

document_key = input_file.split('/')[-2]
url = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv&id={document_key}&gid=0'
response = requests.get(url)
response_content = response.content.decode('utf-8')

In [ ]:
data = StringIO(response_content)
df = pd.read_csv(data, header=1)

In [ ]:
mapped_data = []
for index, row in df.iterrows():
    for col in df.columns[1:]:
        if pd.notna(row[col]) and col != 'średnia' and col != 'miesiąc / rok' and row['miesiąc / rok'] != 'razem':
            mapped_data.append({
                'rok': col,
                'miesiąc': row['miesiąc / rok'],
                'dystans': row[col]
            })

mapped_df = pd.DataFrame(mapped_data)

month_dict = {
    'styczeń': '01', 'luty': '02', 'marzec': '03', 'kwiecień': '04',
    'maj': '05', 'czerwiec': '06', 'lipiec': '07', 'sierpień': '08',
    'wrzesień': '09', 'październik': '10', 'listopad': '11', 'grudzień': '12'}

mapped_df['month'] = mapped_df['miesiąc'].map(month_dict)
mapped_df['date'] = pd.to_datetime(mapped_df['rok'].astype(str) + '-' + mapped_df['miesiąc'].astype(str), format='%Y-%m', errors='coerce')
mapped_df.sort_values(by=['rok', 'month'], inplace=True)
mapped_df.drop(columns=['date', 'month'], inplace=True)

In [ ]:
mapped_df.to_excel(output_file, index=False)